In [3]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, 'plots')

import os

from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets


import json
from sources.SplunkDataset import SplunkDataset
from sources.SplunkFileSource import SplunkFileSource
from SplunkIntel import SplunkIntel


import plotly as py
import pandas as pd
import numpy as np
import plot as plot

py.offline.init_notebook_mode()
pd.set_option('display.notebook_repr_html', True)

data_sources_names = []

data_source_picker = None
start_time_picker = None
end_time_picker = None
test_hosts = None
event_picker = None
event_text_area = None 
threshold_picker = None
new_time_range_picker = None

#create widgets
def create_widgets():
    global data_source_picker, start_time_picker, end_time_picker, event_picker, test_hosts
    global event_text_area, threshold_picker, new_time_range_picker
    for file in os.listdir("../../data_new"):
            if file.endswith("wings.json"):
                data_sources_names.append(file)

    data_source_picker = widgets.Dropdown(
            options=data_sources_names,
            description='Data Source:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    

    start_time_picker = widgets.Dropdown(
            description='Start time:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    

    end_time_picker = widgets.Dropdown(
            description='End time:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    

    test_hosts = widgets.Dropdown(
            description='Test hosts:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    


    
    threshold_picker = widgets.FloatSlider(
                value=0.5,
                min=0,
                max=1,
                step=0.1,
                description='Threshold:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
                slider_color='white',
                color='black'
        )
    
    event_picker = widgets.Dropdown(
            options=[1,2,3],
            description='Event Number:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )
    
    l = widgets.Layout(height='40px', width='800px')
    event_text_area = widgets.Textarea(value='TA: height=40px', layout=l)

    
def initialize(change):    
    # Just to set the widgets
    all_events = []
    all_events.extend(SplunkFileSource.load_data('../../data_new/' + data_source_picker.value))    
    timestamps = set()
    hosts = set()
    for dict in all_events:
            timestamps.add(dict.get('_time'))
            hosts.add(dict.get('host'))

    timestamps = sorted(timestamps)    
    hosts = sorted(hosts)
    
    start_time_picker.options = list(timestamps)
    end_time_picker.options = list(timestamps)
    test_hosts.options = list(hosts)
    
    change_event_handler(None)
    
    
splunkDataset = SplunkDataset()
splunkIntelargs = []

def event_picker_change_handler(change):
    event_text_area.value = json.dumps(splunkDataset.get_all_events()[event_picker.value])

def split(input, length, size):
    input.replace('\n', ' ')
    input.replace('\tat', ' ')
    return '<br>'.join([input[start:start + size] for start in range(0, length, size)])

def get_tool_tips(all_events):
    tooltips = []
    for idx, event in enumerate(all_events):
        tooltips.append(
            split(event[0], min(100, len(event[0])), 100) + '<br> id = ' + str(idx) + '<br> cluster = ' + str(
            event[3]))
            
    return tooltips
    
# Handle data source change
def change_event_handler(change):
    global splunkDataset, test_hosts
    del splunkIntelargs[:]
    clear_output()
    
    print(test_hosts.value)
    splunkDataset.load_from_file_slice_by_nodes('../../data_new/' + data_source_picker.value, test_hosts.value)
    
   
    splunkIntelargs.append('--sim_threshold')
    splunkIntelargs.append(str(float(threshold_picker.value)))
    
    print(splunkIntelargs)

    splunkIntel = SplunkIntel(splunkDataset, SplunkIntel.parse(splunkIntelargs))
    splunkDataset = splunkIntel.run()
    
    plot.scatter_plot(splunkDataset.get_xy_matrix_control(), splunkDataset.get_control_events())

    plot.scatter_plot_groups(splunkDataset.get_xy_matrix_control(), 
                             splunkDataset.get_control_clusters(), 
                             splunkDataset.get_cluster_tags(5))
    
    plot.scatter_plot_groups(splunkDataset.get_xy_matrix_all(), 
                             [event[4] for event in splunkDataset.get_all_events()],
                            get_tool_tips(splunkDataset.get_all_events()), 
                             ['control', 'test', 'test-anomaly'], ['blue','orange','red'])
    
    plot.scatter_plot_groups_4d(splunkDataset.get_xyz_matrix_all(), 
                                [event[5] for event in splunkDataset.get_all_events()],
                                splunkDataset.get_all_clusters(),
                                get_tool_tips(splunkDataset.get_all_events()),
                                ['blue','orange','red'])


            
    event_picker.options = [ x for x in range(len(splunkDataset.get_all_events()))] 
     

create_widgets()

#intialize
initialize(None)
    
#setup even handlers    
#data_source_picker.observe(initialize, names="value")
#time_range_picker.observe(change_event_handler, names="value")
#new_time_range_picker.observe(change_event_handler, names="value")
event_picker.observe(event_picker_change_handler, names="value")
threshold_picker.observe(change_event_handler, names="value")



# show widgets
display(widgets.HBox([data_source_picker]))
display(widgets.HBox([start_time_picker, end_time_picker, threshold_picker]))
#display(widgets.HBox([new_time_range_picker]))
display(widgets.HBox([event_picker, event_text_area]))

ip-172-31-1-210
['--sim_threshold', '0.5']
0
[[u'0' u'2']
 [u'0' u'2']
 [u'0' u'2']]
2
[[u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'3']
 [u'2' u'1']
 [u'2' u'3']
 [u'2' u'3']
 [u'2' u'4']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'3']
 [u'2' u'1']
 [u'2' u'3']
 [u'2' u'3']
 [u'2' u'4']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'3']
 [u'2' u'1']
 [u'2' u'3']
 [u'2' u'3']
 [u'2' u'4']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']
 [u'2' u'1']]
